<a href="https://colab.research.google.com/github/igt-cb/sfdx-scanner/blob/main/panel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# @title Run Once
from io import BytesIO
import numpy as np
import pandas as pd
import holoviews as hv
import panel as pn
!pip install hvplot > /dev/null
from scipy.optimize import minimize
import hvplot.pandas
print ('Imports complete.')

Imports complete.


In [8]:
# @title Pretty Notifications
text = (
    "This is a **{alert_type}** alert with [an example link]"
    "(https://panel.holoviz.org/). Give it a click if you like."
)

pn.Column(*[
    pn.pane.Alert(text.format(alert_type=at), alert_type=at)
    for at in pn.pane.Alert.param.alert_type.objects],
    sizing_mode="stretch_width"
).servable()

Column(design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
    [0] Alert(str, design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
    [1] Alert(str, alert_type='secondary', design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
    [2] Alert(str, alert_type='success', design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
    [3] Alert(str, alert_type='danger', design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
    [4] Alert(str, alert_type='warning', design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
    [5] Alert(str, alert_type='info', design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
    [6] Alert(str, alert_type='light', design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
    [7] Alert(str, alert_type='dark', design=<class 'panel.theme.materi..., sizing_mode='stretch_width')

In [9]:
import hvplot as plot
#@title Template - Portfolio Analyzer
pn.extension('tabulator', design='material', template='material', loading_indicator=True)

@pn.cache
def get_stocks(data):
    if data is None:
        stock_file = 'https://datasets.holoviz.org/stocks/v1/stocks.csv'
    else:
        stock_file = BytesIO(data)
    return pd.read_csv(stock_file, index_col='Date', parse_dates=True)

file_input = pn.widgets.FileInput(sizing_mode='stretch_width')

stocks = hvplot.bind(get_stocks, file_input).interactive()

selector = pn.widgets.MultiSelect(
    name='Select stocks', sizing_mode='stretch_width',
    options=list(stocks.columns.to_list())
)

selected_stocks = stocks.pipe(
    lambda df, cols: df[cols] if cols else df, selector
)
@pn.cache
def find_best_allocation(log_return, vol, ret):
    cols = log_return.shape[1]
    vol = vol or 0
    ret = ret or 0
    mean_return = np.nanmean(log_return, axis=0)
    bounds = tuple((0, 1) for i in range(cols))
    init_guess = [1./cols for i in range(cols)]
    cons = (
        {'type':'eq','fun': check_sum},
        {'type':'eq','fun': lambda w: get_return(mean_return, w) - ret},
        {'type':'eq','fun': lambda w: get_volatility(log_return, w) - vol}
    )
    opt = minimize(
        minimize_difference, init_guess, args=(vol, ret, log_return, mean_return),
        bounds=bounds, constraints=cons
    )
    ret = get_return(mean_return, opt.x)
    vol = get_volatility(log_return, opt.x)
    return pd.Series(list(opt.x)+[ret, vol], index=list(log_return.columns)+['Return', 'Volatility'], name='Weight')

def compute_random_allocations(log_return, num_ports=15000):
    _, ncols = log_return.shape

    # Compute log and mean return
    mean_return = np.nanmean(log_return, axis=0)

    # Allocate normalized weights
    weights = np.random.random((num_ports, ncols))
    normed_weights = (weights.T / np.sum(weights, axis=1)).T
    data = dict(zip(log_return.columns, normed_weights.T))

    # Compute expected return and volatility of random portfolios
    data['Return'] = expected_return = np.sum((mean_return * normed_weights) * 252, axis=1)
    return_covariance = np.cov(log_return[1:], rowvar=False) * 252
    if not return_covariance.shape:
        return_covariance = np.array([[252.]])
    data['Volatility'] = volatility = np.sqrt((normed_weights * np.tensordot(return_covariance, normed_weights.T, axes=1).T).sum(axis=1))
    data['Sharpe'] = sharpe_ratio = expected_return/volatility

    df = pd.DataFrame(data)
    df.attrs['mean_return'] = mean_return
    df.attrs['log_return'] = log_return
    return df

def check_sum(weights):
    return np.sum(weights) - 1

def get_return(mean_ret, weights):
    return np.sum(mean_ret * weights) * 252

def get_volatility(log_ret, weights):
    return np.sqrt(np.dot(weights.T, np.dot(np.cov(log_ret[1:], rowvar=False) * 252, weights)))

def compute_frontier(df, n=30):
    frontier_ret = np.linspace(df.Return.min(), df.Return.max(), n)
    frontier_volatility = []

    cols = len(df.columns) - 3
    bounds = tuple((0, 1) for i in range(cols))
    init_guess = [1./cols for i in range(cols)]
    for possible_return in frontier_ret:
        cons = (
            {'type':'eq', 'fun': check_sum},
            {'type':'eq', 'fun': lambda w: get_return(df.attrs['mean_return'], w) - possible_return}
        )
        result = minimize(lambda w: get_volatility(df.attrs['log_return'], w), init_guess, bounds=bounds, constraints=cons)
        frontier_volatility.append(result['fun'])
    return pd.DataFrame({'Volatility': frontier_volatility, 'Return': frontier_ret})

def minimize_difference(weights, des_vol, des_ret, log_ret, mean_ret):
    ret = get_return(mean_ret, weights)
    vol = get_volatility(log_ret, weights)
    return abs(des_ret-ret) + abs(des_vol-vol)

@pn.cache
def find_best_allocation(log_return, vol, ret):
    cols = log_return.shape[1]
    vol = vol or 0
    ret = ret or 0
    mean_return = np.nanmean(log_return, axis=0)
    bounds = tuple((0, 1) for i in range(cols))
    init_guess = [1./cols for i in range(cols)]
    cons = (
        {'type':'eq','fun': check_sum},
        {'type':'eq','fun': lambda w: get_return(mean_return, w) - ret},
        {'type':'eq','fun': lambda w: get_volatility(log_return, w) - vol}
    )
    opt = minimize(
        minimize_difference, init_guess, args=(vol, ret, log_return, mean_return),
        bounds=bounds, constraints=cons
    )
    ret = get_return(mean_return, opt.x)
    vol = get_volatility(log_return, opt.x)
    return pd.Series(list(opt.x)+[ret, vol], index=list(log_return.columns)+['Return', 'Volatility'], name='Weight')

n_samples = pn.widgets.IntSlider(
    name='Random samples', value=10_000, start=1000, end=20_000, step=1000, sizing_mode='stretch_width'
)

button = pn.widgets.Button(name='Run Analysis', sizing_mode='stretch_width')
posxy = hv.streams.Tap(x=None, y=None)

text = """
#  Portfolio optimization

This application performs portfolio optimization given a set of stock time series.

To optimize your portfolio:

1. Upload a CSV of the daily stock time series for the stocks you are considering
2. Select the stocks to be included.
3. Run the Analysis
4. Click on the Return/Volatility plot to select the desired risk/reward profile

Upload a CSV containing stock data:
"""

explanation = """
The code for this app was taken from [this excellent introduction to Python for Finance](https://github.com/PrateekKumarSingh/Python/tree/master/Python%20for%20Finance/Python-for-Finance-Repo-master).
To learn some of the background and theory about portfolio optimization see [this notebook](https://github.com/PrateekKumarSingh/Python/blob/master/Python%20for%20Finance/Python-for-Finance-Repo-master/09-Python-Finance-Fundamentals/02-Portfolio-Optimization.ipynb).
"""

sidebar = pn.layout.WidgetBox(
    pn.pane.Markdown(text, margin=(0, 10)),
    file_input,
    selector,
    n_samples,
    explanation,
    max_width=350,
    sizing_mode='stretch_width'
).servable(area='sidebar')

sidebar
n_samples = pn.widgets.IntSlider(
    name='Random samples', value=10_000, start=1000, end=20_000, step=1000, sizing_mode='stretch_width'
)
button = pn.widgets.Button(name='Run Analysis', sizing_mode='stretch_width')
posxy = hv.streams.Tap(x=None, y=None)

text = """
#  Portfolio optimization

This application performs portfolio optimization given a set of stock time series.

To optimize your portfolio:

1. Upload a CSV of the daily stock time series for the stocks you are considering
2. Select the stocks to be included.
3. Run the Analysis
4. Click on the Return/Volatility plot to select the desired risk/reward profile

Upload a CSV containing stock data:
"""

explanation = """
The code for this app was taken from [this excellent introduction to Python for Finance](https://github.com/PrateekKumarSingh/Python/tree/master/Python%20for%20Finance/Python-for-Finance-Repo-master).
To learn some of the background and theory about portfolio optimization see [this notebook](https://github.com/PrateekKumarSingh/Python/blob/master/Python%20for%20Finance/Python-for-Finance-Repo-master/09-Python-Finance-Fundamentals/02-Portfolio-Optimization.ipynb).
"""

plot = (allocations_scatter * frontier_curve * max_sharpe_point * closest_point).opts(min_height=400, show_grid=True)

# Set up data pipelines
log_return = np.log(selected_stocks/selected_stocks.shift(1))
random_allocations = log_return.pipe(compute_random_allocations, n_samples)
closest_allocation = log_return.pipe(find_best_allocation, posxy.param.x, posxy.param.y)
efficient_frontier = random_allocations.pipe(compute_frontier)
max_sharpe = random_allocations.pipe(lambda df: df[df.Sharpe==df.Sharpe.max()])

# Generate plots
opts = {'x': 'Volatility', 'y': 'Return', 'responsive': True}

allocations_scatter = random_allocations.hvplot.scatter(
    alpha=0.1, color='Sharpe', cmap='plasma', **opts
).dmap().opts(tools=[])

frontier_curve = efficient_frontier.hvplot(
    line_dash='dashed', color='green', **opts
).dmap()

max_sharpe_point = max_sharpe.hvplot.scatter(
    line_color='black', size=50, **opts
).dmap()

closest_point = closest_allocation.to_frame().T.hvplot.scatter(color='green', line_color='black', size=50, **opts).dmap()

posxy.source = allocations_scatter

summary = pn.pane.Markdown(
    pn.bind(lambda p: f"""
    The selected portfolio has a volatility of {p.Volatility:.2f}, a return of {p.Return:.2f}
    and Sharpe ratio of {p.Return/p.Volatility:.2f}.""", closest_allocation), width=250
)

table = pn.widgets.Tabulator(closest_allocation.to_frame().iloc[:-2])


pn.Row(plot, pn.Column(summary, table), sizing_mode='stretch_both')
investment = pn.widgets.Spinner(name='Investment Value in $', value=5000, step=1000, start=1000, end=100000)
year = pn.widgets.DateRangeSlider(name='Year', value=(stocks.index.min().eval(), stocks.index.max().eval()), start=stocks.index.min(), end=stocks.index.max())

stocks_between_dates = selected_stocks[year.param.value_start:year.param.value_end]
price_on_start_date = selected_stocks[year.param.value_start:].iloc[0]
allocation = (closest_allocation.iloc[:-2] * investment)

performance_plot = (stocks_between_dates * allocation / price_on_start_date).sum(axis=1).rename().hvplot.line(
    ylabel='Total Value ($)', title='Portfolio performance', responsive=True, min_height=400
).dmap()

performance = pn.Column(
    pn.Row(year, investment),
    performance_plot,
    sizing_mode='stretch_both'
)
timeseries = selected_stocks.hvplot.line(
    'Date', group_label='Stock', value_label='Stock Price ($)', title='Daily Stock Price',
    min_height=300, responsive=True, grid=True, legend='top_left'
).dmap()

log_ret_hists = log_return.hvplot.hist(min_height=300, min_width=400, responsive=True, bins=100, subplots=True, group_label='Stock').cols(2).opts(sizing_mode='stretch_both').panel()

main = pn.Tabs(
    ('Analysis', pn.Column(
            pn.Row(
                plot, pn.Column(summary, table),
                sizing_mode='stretch_both'
            ),
            performance,
            sizing_mode='stretch_both'
        )
    ),
    ('Timeseries', timeseries),
    ('Log Return', pn.Column(
        '## Daily normalized log returns',
        'Width of distribution indicates volatility and center of distribution the mean daily return.',
        log_ret_hists,
        sizing_mode='stretch_both'
    )),
    sizing_mode='stretch_both', min_height=800
).servable(title='Portfolio Optimizer')

pn.Card(pn.Row(sidebar, main)).servable()

/usr/local/lib/python3.10/dist-packages/holoviews/plotting/bokeh/plot.py:967: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  layout_plot = gridplot(
/usr/local/lib/python3.10/dist-packages/holoviews/plotting/bokeh/plot.py:967: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  layout_plot = gridplot(


Card(design=<class 'panel.theme.materi...)
    [0] Row(design=<class 'panel.theme.materi...)
        [0] WidgetBox(design=<class 'panel.theme.materi..., max_width=350, sizing_mode='stretch_width')
            [0] Markdown(str, design=<class 'panel.theme.materi..., margin=(0, 10))
            [1] FileInput(design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
            [2] MultiSelect(design=<class 'panel.theme.materi..., name='Select stocks', options=[<hvplot.interactive.Inter...], sizing_mode='stretch_width')
            [3] IntSlider(design=<class 'panel.theme.materi..., end=20000, name='Random samples', sizing_mode='stretch_width', start=1000, step=1000, value=10000)
            [4] Markdown(str, design=<class 'panel.theme.materi...)
        [1] Tabs(design=<class 'panel.theme.materi..., min_height=800, sizing_mode='stretch_both')
            [0] Column(design=<class 'panel.theme.materi..., sizing_mode='stretch_both')
                [0] Row(design=<class 'panel.theme.materi..., sizing_mode='stretch_both')
                    [0] HoloViews(DynamicMap, design=<class 'panel.theme.materi..., sizing_mode='stretch_both')
                    [1] Column(design=<class 'panel.theme.materi...)
                        [0] Markdown(str, design=<class 'panel.theme.materi..., width=250)
                        [1] Tabulator(design=<class 'panel.theme.materi..., value=         Weight
aapl  ...)
                [1] Column(design=<class 'panel.theme.materi..., sizing_mode='stretch_both')
                    [0] Row(design=<class 'panel.theme.materi...)
                        [0] DateRangeSlider(design=<class 'panel.theme.materi..., end=Timestamp('2016-12-30 0..., name='Year', start=Timestamp('2012-01-03 0..., value=(Timestamp('2012-01-03 00:..., value_end=Timestamp('2016-12-30 0..., value_start=Timestamp('2012-01-03 0...)
                        [1] IntInput(design=<class 'panel.theme.materi..., end=100000, name='Investment Value in $', start=1000, step=1000, value=5000)
                    [1] HoloViews(DynamicMap, design=<class 'panel.theme.materi..., sizing_mode='stretch_both')
            [1] HoloViews(DynamicMap, design=<class 'panel.theme.materi..., name='Timeseries', sizing_mode='stretch_both')
            [2] Column(design=<class 'panel.theme.materi..., sizing_mode='stretch_both')
                [0] Markdown(str, design=<class 'panel.theme.materi...)
                [1] Markdown(str, design=<class 'panel.theme.materi...)
                [2] ParamFunction(function, _pane=HoloViews, defer_load=False, design=<class 'panel.theme.materi...)

In [10]:
# @title Template Grid { run: "auto", vertical-output: true }
import pandas as pd
import numpy as np
import panel as pn

from bokeh.plotting import figure

from scipy.optimize import minimize

from panel.interact import interact
from panel import widgets

pn.extension()

# Define the grid layout and its sizes
grid = pn.GridSpec(sizing_mode="stretch_both")

# Create the dashboard components
title = pn.pane.Markdown("# Dashboard Title",styles={'background': 'white'}, sizing_mode="stretch_width")

sidebar = pn.Column(
    pn.pane.Markdown("""
    ## Sidebar

    * Item 1
    * Item 2
    * Item 3
    """
    ),
    sizing_mode="fixed", width=200
)

overview = pn.Column(
    pn.pane.Markdown("""
    ## Overview

    Some important metrics and summaries go here.
    """
    ),
    sizing_mode="stretch_height"
)

summary = pn.pane.Markdown("""
## Summary

Some detailed metrics and data go here.
""", sizing_mode="stretch_both")

chart = pn.pane.Markdown("""
## Chart

Some visualizations or charts go here.
""", sizing_mode="stretch_both")

metrics = pn.Column(
    sizing_mode="stretch_height"
)

footer = pn.pane.Markdown("© 2022 Dashboard Inc.",styles={'background': 'lightgray'} )

# Add components to the grid layout
grid[0, 0:2] = title
grid[1, 0] = sidebar
grid[1, 1] = overview
grid[2, 0] = pn.layout.HSpacer()
grid[2, 1] = chart
grid[3, 0] = pn.layout.HSpacer()
grid[3, 1] = summary
grid[4, 0:2] = footer

pn.extension()
pn.Card(grid,height=500).servable()


Card(design=<class 'panel.theme.materi..., height=500)
    [0] GridSpec(design=<class 'panel.theme.materi..., ncols=2, nrows=5, sizing_mode='stretch_both')
        [0] Markdown(str, design=<class 'panel.theme.materi..., sizing_mode='stretch_both', styles={'background': 'white'})
        [1] Column(design=<class 'panel.theme.materi..., sizing_mode='stretch_both', width=200)
            [0] Markdown(str, design=<class 'panel.theme.materi...)
        [2] Column(design=<class 'panel.theme.materi..., sizing_mode='stretch_both')
            [0] Markdown(str, design=<class 'panel.theme.materi...)
        [3] HSpacer(design=<class 'panel.theme.materi...)
        [4] Markdown(str, design=<class 'panel.theme.materi..., sizing_mode='stretch_both')
        [5] HSpacer(design=<class 'panel.theme.materi...)
        [6] Markdown(str, design=<class 'panel.theme.materi..., sizing_mode='stretch_both')
        [7] Markdown(str, design=<class 'panel.theme.materi..., sizing_mode='stretch_both', styles={'background': '...})

In [11]:
# @title Default title text
import panel as pn
pn.extension()


def cardi_b(existing_pn, collection=[]):
  return pn.Card(existing_pn)

def cardify_all(the_pn_elements_array):
  allcards=[]
  for elem in the_pn_elements_array:
    allcards.append(cardi_b(elem))
  return allcards

def get_code_editor():
  import panel as pn
  import param

  pn.extension('codeeditor')
  py_code = "import sys"
  return pn.widgets.CodeEditor(value=py_code, sizing_mode='stretch_width', language='python', height=300)

def get_checkbox_group():
  return pn.widgets.CheckBoxGroup(
    name='Checkbox Group', value=['Apple', 'Pear'], options=['Apple', 'Banana', 'Pear', 'Strawberry'],
    inline=True)

def get_chatbox():
  chat_box = pn.widgets.ChatBox(
    value=[
        {"You": "Hello!"},
        {"Machine": "Greetings!"},
        {"You": "Question for you..."},
        {"You": "What is the meaning of life?"},
        {"Machine": "I don't know. Do you know?"},
    ],
  )
  return chat_box

def get_autocomplete(options=['Biology', 'Chemistry', 'Physics']):
    return pn.widgets.AutocompleteInput(name='Autocomplete Input')

def perform_operation(operation):
  return operation()


operations=[get_code_editor,get_chatbox,get_autocomplete,get_checkbox_group]
final_layout_items=[]
final_layout_items.append(perform_operation(operation) for operation in operations)
final_layout_items = cardify_all(final_layout_items)

pn.extension("bokeh")
appp=None
apps=pn.Row([*final_layout_items])
apps.servable()

Row(design=<class 'panel.theme.materi...)
    [0] Str(list, design=<class 'panel.theme.materi...)